# ElasticSearch and StarWars

Based on: https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/

albertoueda - 2016/09/29 

In [103]:
# Simplest connection check
import requests
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "Alberto",\n  "cluster_name" : "elasticsearch",\n  "version" : {\n    "number" : "2.4.0",\n    "build_hash" : "ce9f0c7394dee074091dd1bc4e9469251181fc55",\n    "build_timestamp" : "2016-08-29T09:14:17Z",\n    "build_snapshot" : false,\n    "lucene_version" : "5.5.2"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [104]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [9]:
#index some test data
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

{'_id': '1',
 '_index': 'test-index',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 1,
 'created': True}

In [10]:
#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)

{'_id': '1',
 '_index': 'test-index',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 2,
 'found': True}

In [11]:
es.index(index='sw', doc_type='people', id=1, body={
	"name": "Luke Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})

{'_id': '1',
 '_index': 'sw',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'people',
 '_version': 1,
 'created': True}

In [110]:
es.get(index='sw', doc_type='people', id=1)

{'_id': '1',
 '_index': 'sw',
 '_source': {'birth_year': '19BBY',
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'eye_color': 'blue',
  'films': ['http://swapi.co/api/films/6/',
   'http://swapi.co/api/films/3/',
   'http://swapi.co/api/films/2/',
   'http://swapi.co/api/films/1/',
   'http://swapi.co/api/films/7/'],
  'gender': 'male',
  'hair_color': 'blond',
  'height': '172',
  'homeworld': 'http://swapi.co/api/planets/1/',
  'mass': '77',
  'name': 'Luke Skywalker',
  'skin_color': 'fair',
  'species': ['http://swapi.co/api/species/1/'],
  'starships': ['http://swapi.co/api/starships/12/',
   'http://swapi.co/api/starships/22/'],
  'url': 'http://swapi.co/api/people/1/',
  'vehicles': ['http://swapi.co/api/vehicles/14/',
   'http://swapi.co/api/vehicles/30/']},
 '_type': 'people',
 '_version': 2,
 'found': True}

In [21]:
#let's iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content.decode('utf-8')))
    i=i+1
 
print(i)

18


In [22]:
es.get(index='sw', doc_type='people', id=5)

{'_id': '5',
 '_index': 'sw',
 '_source': {'birth_year': '19BBY',
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'eye_color': 'brown',
  'films': ['http://swapi.co/api/films/6/',
   'http://swapi.co/api/films/3/',
   'http://swapi.co/api/films/2/',
   'http://swapi.co/api/films/1/',
   'http://swapi.co/api/films/7/'],
  'gender': 'female',
  'hair_color': 'brown',
  'height': '150',
  'homeworld': 'http://swapi.co/api/planets/2/',
  'mass': '49',
  'name': 'Leia Organa',
  'skin_color': 'light',
  'species': ['http://swapi.co/api/species/1/'],
  'starships': [],
  'url': 'http://swapi.co/api/people/5/',
  'vehicles': ['http://swapi.co/api/vehicles/30/']},
 '_type': 'people',
 '_version': 1,
 'found': True}

In [23]:
es.search(index="mag", body={"query": {"match": {'name':'Darth Vader'}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'sw',
    '_score': 1.4965448,
    '_source': {'birth_year': '41.9BBY',
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'eye_color': 'yellow',
     'films': ['http://swapi.co/api/films/6/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/2/',
      'http://swapi.co/api/films/1/'],
     'gender': 'male',
     'hair_color': 'none',
     'height': '202',
     'homeworld': 'http://swapi.co/api/planets/1/',
     'mass': '136',
     'name': 'Darth Vader',
     'skin_color': 'white',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/13/'],
     'url': 'http://swapi.co/api/people/4/',
     'vehicles': []},
    '_type': 'people'}],
  'max_score': 1.4965448,
  'total': 1},
 'timed_out': False,
 'took': 220}

In [24]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '19BBY',
     'created': '2014-12-09T13:50:51.644000Z',
     'edited': '2014-12-20T21:17:56.891000Z',
     'eye_color': 'blue',
     'films': ['http://swapi.co/api/films/6/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/2/',
      'http://swapi.co/api/films/1/',
      'http://swapi.co/api/films/7/'],
     'gender': 'male',
     'hair_color': 'blond',
     'height': '172',
     'homeworld': 'http://swapi.co/api/planets/1/',
     'mass': '77',
     'name': 'Luke Skywalker',
     'skin_color': 'fair',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/12/',
      'http://swapi.co/api/starships/22/'],
     'url': 'http://swapi.co/api/people/1/',
     'vehicles': ['http://swapi.co/api/vehicles/14/',
      'http://swapi.co/api/vehicles/30/']},
    '_type': 'peop

In [86]:
for i in es.search(index="sw", body={"query": {"wildcard" : { "name" : "l*" }}})['hits']['hits'] :
        print(i['_source']['name'])

Leia Organa
Owen Lars
Luke Skywalker
Beru Whitesun lars


In [92]:
for i in es.search(index="sw", body={"query": {"term" : { "name" : "luke" }}})['hits']['hits'] :
        print(i['_source']['name'])

Luke Skywalker


In [108]:
es.index(index='sw', doc_type='people', id=44, body={
	"name": "Alberto Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})

{'_id': '45',
 '_index': 'sw',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'people',
 '_version': 1,
 'created': True}

In [105]:
# Persistent data test. The data must be available after ES restart
for i in es.search(index="sw", body={"query": {"term" : { "name" : "skywalker" }}})['hits']['hits'] :
        print(i['_source']['name'])

Alberto Skywalker
Luke Skywalker
Anakin Skywalker
